In [17]:
import numpy as np
import pandas as pd
import streamlit as st
import csv
from collections import defaultdict
from numpy.random import default_rng as rng

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
from os import path
from matplotlib.pyplot import figure
if 'Sarabun' not in [x.name for x in matplotlib.font_manager.fontManager.ttflist]:
    matplotlib.font_manager.fontManager.addfont('Sarabun-Regular.ttf')
matplotlib.rc('font', family='Sarabun')

from rsi_modules import *

mm3_url         = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=1475905714"
publication_url = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=2125519370"
theses_url      = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=609352385"

F = pd.read_csv(mm3_url)
publication = pd.read_csv(publication_url).fillna("")
theses = pd.read_csv(theses_url).fillna("")
theses = theses.drop_duplicates(subset=['รหัส (นักศึกษา)'], keep='last')

In [18]:
publication,df = get_publication(publication)
df = df.reset_index().rename(columns={'index':'ปี'})
# df.to_csv('publication.csv')
# st.write(df)

2026-02-10 16:53:44.676 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-10 16:53:44.857 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-10 16:53:44.858 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-10 16:53:44.858 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [53]:
NAMES = []
for n in publication.columns:
    if 'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ' in n:
        # print(n)
        NAMES.append(n) 

In [54]:
name_paper = defaultdict(list)
names = ['ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [ศ.ดร.ทวี  เชื้อสุวรรณทวี]','ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [ดร.วิษณุ นิตยธรรมกุล]']

for name in NAMES:
    papers = []
    for i,(role,TCI,Q,conf_int,conf_nat) in enumerate(zip(publication[name],publication['วารสารไทย'],publication['วารสารนานาชาติ'],publication['ประชุมวิชาการระดับนานาชาติ'],publication['ประชุมวิชาการระดับชาติ'])):
        if role in ['First Author', 'Corresponding Author']:
            papers+=[TCI+Q+conf_int+conf_nat]
            # if len(Q) > 0: collect_Q.append(Q)
            # if len(TCI) > 0: collect_TCI.append(TCI)
            # if len(conf_int) > 0: collect_conf_int.append(conf_int)
            # if len(conf_nat) > 0: collect_conf_nat.append(conf_nat)
    name_paper[name] = papers

In [55]:
name_paper

defaultdict(list,
            {'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [นพ.สมเกียรติ ลีละศิธร]': [],
             'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [พญ.วัชรา ริ้วไพบูลย์]': [],
             'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [ศ.ดร.ทวี  เชื้อสุวรรณทวี]': ['Q3',
              'Q2',
              'Q1',
              'TCI1'],
             'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [รศ.ดร.อาดัม นีละไพจิตร]': ['TCI1',
              'อื่นๆ',
              'อื่นๆ',
              'อื่นๆ'],
             'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [รศ.ดร.ณัฏฐนียา โตรักษา]': [],
             'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [ผศ.ดร.อารี ภาวสุทธิไพศิฐ]': [],
             'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [ผศ.ดร.ธีรศักดิ์ ศรีสุรกุล]': [],
             'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [ผศ.ดร.เจนจิรา เจนจิตรวาณิช]': [],
             'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [ดร.รติรส จันทรสมดี]': ['TCI1'],
             'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [ดร.อิศวรา ศิริรุ่งเรือง]': [],
             'ชื่ออาจารย์และสถานะผู้แต่ง

In [21]:
for t in publication.columns:
    if 'วารสารไทย (TCI1-TCI2)' in t: publication.rename(columns={t:'วารสารไทย'}, inplace=True)
    elif 'วารสารนานาชาติ (Q1-Q4 หรืออื่นๆ)' in t: publication.rename(columns={t:'วารสารนานาชาติ'}, inplace=True)
    elif 'ประชุมวิชาการระดับนานาชาติ' in t: publication.rename(columns={t:'ประชุมวิชาการระดับนานาชาติ'}, inplace=True)
    elif 'ประชุมวิชาการระดับชาติ' in t: publication.rename(columns={t:'ประชุมวิชาการระดับชาติ'}, inplace=True)


In [ ]:
publication['วารสารไทย'],publication['วารสารนานาชาติ'],publication['ประชุมวิชาการระดับนานาชาติ'],publication['ประชุมวิชาการระดับชาติ']

KeyError: ('วารสารไทย', 'วารสารนานาชาติ', 'ประชุมวิชาการระดับนานาชาติ', 'ประชุมวิชาการระดับชาติ')

In [ ]:
st.title('ภาระงานของคณาจารย์ในสถาบันราชสุดาปีการศึกษา 2568')

st.subheader("1. ชั่วโมงสอนและจำนวนวิชาที่รับผิดชอบ")

academic_year = st.radio(
    "เลือกเทอมและปีการศึกษา",
    ["2568/1", "2568/2"],
)

if academic_year == "2568/1":
    start_date = '8/5/2025'
    end_date   = '9/19/2025'
elif academic_year == "2568/2":
    start_date = '12/12/2025'
    end_date   = '12/30/2025'

f = F[F['Timestamp'].between(start_date, end_date)]
t1,t2 = report_mm3(f)

st.subheader("2. ผลงานตีพิมพ์ทางวิชาการในแต่ละปี")
publication,df = get_publication(publication)
df = df.reset_index().rename(columns={'index':'ปี'})
st.write(df)

st.subheader("3. ภาระงานควบคุมวิทยานิพนธ์")
B = pd.DataFrame(get_thesis_workload_new(theses)).T
st.write(B)

st.subheader("4. ภาพรวมความก้าวหน้าวิทยานิพนธ์ของนักศึกษา")
N = get_thesis_progression_summary(theses)
A = pd.concat(N, axis=1)
st.write(A)
st.write("หมายเหตุ:")
st.write("MA  = หส.ปริญญาโท สาขาคุณภาพชชีวิตคนพิการ")
st.write("MEd = หส.ปริญญาโท สาขาการศึกษาพิเศษ")
st.write("PhD = หส.ปรัชญาดุษฎีบัณฑิต สาขาคุณภาพชีวิตคนพิการ")

2025-12-24 13:56:05.508 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.340 
  command:

    streamlit run c:\Users\User\miniconda3\envs\rsi_dashboard\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-24 13:56:06.341 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.342 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.343 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.344 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.344 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.345 Thre

In [2]:
N = get_thesis_progression_summary(theses)
A = pd.concat(N, axis=1)

In [3]:
st.write(A)

2025-12-17 17:05:31.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-17 17:05:31.468 
  command:

    streamlit run c:\Users\User\miniconda3\envs\rsi_dashboard\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-17 17:05:31.468 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-17 17:05:31.470 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
